In [2]:
import numpy as np
import open3d as o3d
from PointClouds.Registration import mstorePCR

view_count = 16
point_data = "blocks"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Direct Point Cloud Normal Calculation

In [3]:
pcd = o3d.io.read_point_cloud(f"../RandomPushing/data/pcs/{point_data}/mstore.pcd")
pcd = pcd.voxel_down_sample(voxel_size=0.005)
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.025, max_nn=20)
)
pcd.orient_normals_consistent_tangent_plane(k=30)

# Visualize the point cloud with smoothed normals
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

### Better Point Cloud Normal Calculation Method
- Loop through all camera views
- Rotate the views to get a better normal direction estimation (Can still be improved)
- Estimate the point normals
- Combine the point clouds with the mstore function
- Downsample the points and use a voting system for averaging the normals

In [5]:
step = -2.*np.pi/float(view_count)

for i in range(view_count):
    pcd = o3d.io.read_point_cloud(f"../RandomPushing/data/pcs/{point_data}/point_cloud_{i}.pcd")
    pcd = pcd.voxel_down_sample(voxel_size=.001)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Center point cloud
    min_coor = np.array([
        min([p[0] for p in pcd.points]),
        min([p[1] for p in pcd.points]),
        min([p[2] for p in pcd.points])
    ])

    max_coor = np.array([
        max([p[0] for p in pcd.points]),
        max([p[1] for p in pcd.points]),
        max([p[2] for p in pcd.points])
    ])

    midpoint = (max_coor+min_coor)*.5

    for j in range(len(pcd.points)):
        pcd.points[j] -= midpoint
        tmp = pcd.points[j][1]
        pcd.points[j][1] = pcd.points[j][2]
        pcd.points[j][2] = tmp

    # Rotate point cloud
    angle = i*step
    rotation_matrix = np.array([
        [np.cos(angle), 0., np.sin(angle)],
        [0., 1., 0.],
        [-np.sin(angle), 0., np.cos(angle)]
    ])
    for j in range(len(pcd.points)):
        pcd.points[j] = rotation_matrix @ pcd.points[j]

    # Calculate and display normals
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.025, max_nn=20))
    
    pcd.orient_normals_consistent_tangent_plane(k=30)
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)

    # Rotate back (king of ugly)
    angle = -i*step
    rotation_matrix = np.array([
        [np.cos(angle), 0., np.sin(angle)],
        [0., 1., 0.],
        [-np.sin(angle), 0., np.cos(angle)]
    ])
    for j in range(len(pcd.points)):
        pcd.points[j] = rotation_matrix @ pcd.points[j]

Combining Process